# Cerebral Cortex ECG and RR Processing on the WESAD data set

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Check notebook dependencies

In [2]:
import importlib, sys, os

sys.path.insert(0, os.path.abspath('..'))
IN_COLAB = 'google.colab' in sys.modules
JAVA_HOME_DEFINED = "JAVA_HOME" in os.environ
SPARK_HOME_DEFINED = "SPARK_HOME" in os.environ
PYSPARK_PYTHON_DEFINED = "PYSPARK_PYTHON" in os.environ
PYSPARK_DRIVER_PYTHON_DEFINED = "PYSPARK_DRIVER_PYTHON" in os.environ
HAVE_MFLOW = importlib.util.find_spec("mFlow") is not None

if(not HAVE_MFLOW):
    if IN_COLAB: 
        print("\nGetting mFlow")
        !git clone https://github.com/mlds-lab/mFlow.git
        !pip install ./mFlow
    else:
        raise SystemExit("Please install mFlow from https://github.com/mlds-lab/mFlow.git")
else:
    print("mFlow found")
    
if not JAVA_HOME_DEFINED:
    if IN_COLAB: 
        print("\nGetting Java 8 SDK")
        !sudo apt update
        !apt-get install -y openjdk-8-jdk-headless graphviz
        os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
    else:
        raise SystemExit("Please install the Java8 SDK and set the JAVA_HOME environment variable")
else:
    print("JAVA_HOME defined")


if not SPARK_HOME_DEFINED:
    if IN_COLAB:
        print("\nGetting Apache Spark 3.1.2")
        !wget  https://mirrors.ocf.berkeley.edu/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
        !tar xf spark-3.1.2-bin-hadoop2.7.tgz
        os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7/"
    else:
        raise SystemExit("Please install spark-3.1.2-bin-hadoop2.7 and set the SPARK_HOME environment variable")
else:
    print("SPARK_HOME defined")


if not PYSPARK_PYTHON_DEFINED:
    if IN_COLAB:
        os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
    else:
        raise SystemExit("Please set the PYSPARK_PYTHON environment variable to your desired Python version")
else:
    print("PYSPARK_PYTHON defined")
            
if not PYSPARK_DRIVER_PYTHON_DEFINED:
    if IN_COLAB:
        os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
    else:
        raise SystemExit("Please set the PYSPARK_DRIVER_PYTHON environment variable to your desired Python version")
else:
    print("PYSPARK_DRIVER_PYTHON defined")            
    

mFlow found
JAVA_HOME defined
SPARK_HOME defined
PYSPARK_PYTHON defined
PYSPARK_DRIVER_PYTHON defined


## Import library functions

In [3]:
#Import mFlow Functions
from mFlow.Blocks.data_loader_wesad import wesad_data_loader
from mFlow.Blocks.ccwrapper import ccwrapper, cc_to_pandas
from mFlow.Workflow.workflow import workflow

#Import cc3 functions
from cerebralcortex.kernel import Kernel
from cerebralcortex.algorithms.ecg.autosense_data_quality import ecg_autosense_data_quality
from cerebralcortex.algorithms.ecg.autosense_rr_interval import get_rr_interval
from cerebralcortex.algorithms.ecg.hrv_features import get_hrv_features


## Define and execute the workflow

In [ ]:
#Define data set properties
Fs         = 700
sensor     = "respiban"

#Define Workflow
dsd        = wesad_data_loader(data_size="small")
dfl        = cc_to_pandas(dsd,name="labels_to_pandas",key="labels",participant_field="user",datetime_field="timestamp",cache_filename="wesad_labels_small.pkl")
dsd        = ccwrapper(ecg_autosense_data_quality, dsd, sort_field="timestamp", sensor_name=sensor, Fs=Fs)
df_ecg     = cc_to_pandas(dsd,name="ecg_to_pandas",participant_field="user",datetime_field="timestamp",time_trunc=None,cache_filename="wesad_ecg_small.pkl")
dsd        = ccwrapper(get_rr_interval,dsd, sort_field="timestamp",Fs=Fs)
df_rr      = cc_to_pandas(dsd,name="rr_to_pandas",participant_field="user",datetime_field="timestamp",time_trunc=None,cache_filename="wesad_rr_small.pkl")

#Execute workflow
flow       = workflow({"ecg":df_ecg,"rr":df_rr,"labels":dfl});
output     = flow.run(backend="sequential", monitor=True, from_scratch=True);


## Display the results

In [ ]:
display(output["ecg"]["dataframe"])

In [ ]:
display(output["rr"]["dataframe"])

In [ ]:
display(output["labels"]["dataframe"])